
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [193]:
import pandas as pd
import numpy as np
from scipy import stats

In [194]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [195]:
# number of callbacks for balck-sounding names
sum(data[data.race=='b'].call)

157.0

In [196]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [197]:
# dataframe for black sounding names

black = data[data.race == 'b']
white = data[data.race == 'w']

In [151]:
black.call.describe()

count    2435.000000
mean        0.064476
std         0.245649
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [152]:
white.call.describe()

count    2435.000000
mean        0.096509
std         0.295346
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

# Question # 1:  What test is appropriate for this problem? Does CLT apply?

* No, the Central Limit Theorem (CTL) does no apply, because this is a problem of independant categorical values, therefore a Chi-Squared test is appropriate. 

# Question # 2:  What are the null and alternate hypotheses?

* Ho: Getting calls for job interviews is independant of race
* Ha: Getting calls for job interviews is dependant on race

Confidence Level = 95%, thus alpha = 0.05.

If p < .05, reject Ho


# Question # 3: Compute margin of error, confidence interval, and p-value.

In [153]:
black_called = len(black[black['call'] == True])
black_not_called = len(black[black['call'] == False])
white_called = len(white[white['call'] == True])
white_not_called = len(white[white['call'] == False])

In [206]:
observed = pd.DataFrame({'black': {'called': black_called, 'not_called': black_not_called},
                         'white': {'called' : white_called, 'not_called' : white_not_called}})

In [219]:
observed

,black,white
called,157,235
not_called,2278,2200


# Compute the margin of error(chi-squared statistic) and p-value:

The below function shows:
* the chi-squared statistic of 16.449
* the p-value of 4.9975e-05
* the array of expected values is 196 blacks called, 196 whites called, 2239 blacks not called, 2239 whites not called

In [221]:
stats.chi2_contingency(observed= observed)

(16.449028584189371, 4.9975783899632552e-05, 1, array([[  196.,   196.],
        [ 2239.,  2239.]]))

In [223]:
chi_squared_stat = 16.449028584189371

# Compute the confidence interval:

* Similar to the t-test where we compared the t-test statistic to a critical value based on the t-distribution to determine whether the result is significant, in the chi-square test we compare the chi-square test statistic to a critical value based on the chi-square distribution. The scipy library shorthand for the chi-square distribution is chi2.
* I assume a 95% confidence level with 1 degree of fredom (The degrees of freedom for a test of independence equals the product of the number of categories in each variable minus 1. In this case we have a 2x2 table so df = 1x1 = 1.
* The critical value is 3.84 and the p-value is 4.9976e-05 (the same as calculated above)
* **Since the chi-squared statistic of 16.449 exceeds the critical value of 3.84, we should reject the null hypothesis that getting calls for job interviews is independant of race.**
* In additon the p-value of 4.9976e-05 is less than the 0.05 confidence interval, thus we should reject the null hypothesis.

In [224]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 1)   # Df = number of variable categories - 1

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=1)
print("P value")
print(p_value)

Critical value
3.84145882069
P value
4.99757838996e-05


# Question # 4: Discuss statistical significance
* Statistical significance (or a statistically significant result) is attained whenever the observed p-value of a test statistic is less than the significance level defined for the hypothesis test.

Above we chose to use 95% confidence, therefore alpha is 0.05.  The p-value of the chi-squared statistic above was 4.99767e-05, which is less than the alpha of 0.05.  Therefore, the above hypothesis test attained a statistically significant result (a result that is not likely to occur randomly, but rather is likely to be attributable to a specific cause.)

